<br>

# Agenda 

### 1.   Announcement
### 2.   Review of the first lecture
### 3.   Outline of the Course
### 4.   Brief Introduction to Python
### 5.   Writing simple screener in Python




## 1. Announcement

* ##   Three Hour Class
* ##   Office hours on Demand




## 2.Review the first lecture
* ###  Context of Algorithmic Trading - Basic Drivers
* ###  Uptick in Volume and Liquidity
* ###  Tighter spread on Products
* ###  Increase in Transperancy of Trading Activities
* ###  Provide higher value to investor and issuers.































##3. Markets and Trading 
   * ###  Buy Side Vs Sell Side
   * ###  Market Makers Vs Liquidity Seekers
   * ###  Systemic Vs Non-Systemic
   * ###  Trading Strategies
      * ### Model Based Strategies
      * ### Machine learning Model Based Strategy

## 4. Outline of the Course
* ### Lecture 3 and  Lecture 4 
   * ### Trend Following and Mean Reversion Stratgies Strategies  
   * ### Relative Value / Pairs Trading Strategies
   * ### Time series Analysis - ARIMA model
   * ### Long Short Equity using - Garch Model
* ### Lecture 5 
   * ### Machine Learning basics with Python
* ### Lecture 6 and 7
   * ### Tensor Flow pipelines for strategies 
   * ### Neural Networks using Tensor Flow
* ### Lecture 8
   * ### Renforcement learning based Trading Strategy  
